In [1]:
from burstextractor.burstlist import download_burst_data
from burstextractor.timeutils import extract_time, fix_typos_in_time, fix_24_hour_time, create_datetime
from burstextractor.data_utils import explode_instruments_long_clean_instruments, keep_only_type_I_to_VI
import pandas as pd
import numpy as np
import difflib

## Create Dataframe with the Burstlists

In [4]:
burst_list = download_burst_data([2021, 2022, 2023], months=range(1, 11), folder="ecallisto_files")

/home/kenfus/Radio-sunburst-labelcreation/burstextractor/burstlist.py:39: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  data = pd.read_csv(


In [5]:
burst_list

,date,time,type,instruments
0,20210119,02:42-02:42,III,Australia-ASSA
1,20210120,12:37-12:37,III,"AUSTRIA-UNIGRAZ, [HUMAIN], MRT1, SOUTHAFRICA-S..."
2,20210127,04:32-04:32,III,"Australia-ASSA, INDIA-GAURI, SOUTHAFRICA-SANSA"
3,20210127,09:27-09:27,III,"AUSTRIA-UNIGRAZ, INDIA-GAURI, INDIA-OOTY, MRT1..."
4,20210218,18:04-18:04,III,"GREENLAND, MEXART, ROSWELL-NM"
...,...,...,...,...
6910,20231031,15:06-15:06,III,"GLASGOW, HUMAIN"
6911,20231031,19:25-19:25,RBR,MEXART
6912,20231031,20:20-20:22,III,"ALASKA-COHOE, ALASKA-HAARP, Australia-ASSA, ME..."
6913,20231031,21:04-21:13,II,"ALASKA-COHOE, ALASKA-HAARP, Australia-ASSA, ME..."


In [6]:
burst_list.sample(5)

,date,time,type,instruments
6528,20230806,07:46-07:46,III,"ALMATY, AUSTRIA-UNIGRAZ, DENMARK, EGYPT-Alexan..."
5904,20230623,12:41-12:52,VI,"ALGERIA-CRAAG, AUSTRIA-UNIGRAZ, Arecibo-Observ..."
3577,20220924,00:47-00:47,III,"ALASKA-COHOE, ALASKA-HAARP, Australia-ASSA"
4851,20230326,14:41-14:42,III,"AUSTRIA-UNIGRAZ, Arecibo-Observatory, BIR, GER..."
5098,20230420,17:21-17:21,III,"ALASKA-COHOE, ALASKA-HAARP, Arecibo-Observator..."


In [7]:
burst_list.shape

(6915, 4)

## Fix typos

In [8]:
extracted_digits = burst_list['time'].str.extract(r'(\d+).(\d+).(\d+).(\d+)', expand=True).astype(int)

In [9]:
impossible_times_bool = (extracted_digits[1] > 59) | (extracted_digits[3] > 59) | (extracted_digits[0] > 23) | (extracted_digits[2] > 23)
extracted_digits[impossible_times_bool]

,0,1,2,3
1436,6,6,6,88
1981,24,32,14,33
3619,23,59,24,0
3633,0,0,24,0
3663,3,50,24,0
5394,21,18,212,19


In [10]:
burst_list[impossible_times_bool]

,date,time,type,instruments
1436,20220210,06:06-06:88,V,"ALMATY, Australia-ASSA, INDIA-OOTY, INDIA-UDAI..."
1981,20220421,24:32-14:33,III,"AUSTRIA-MICHELBACH, Arecibo-Observatory, GLASG..."
3619,20220926,23:59:24:00,III,"ALASKA-COHOE, ALASKA-HAARP, Australia-ASSA"
3633,20220929,00:00-24:00,NaN,CTM
3663,20221001,03:50-24:00,CTM,*
5394,20230510,21:18-212:19,III,"(ALASKA-ANCHORAGE), ALASKA-COHOE, ALASKA-HAARP..."


In [11]:
burst_list.loc[4179]

date                       20230124
time                    03:24-03:27
type                             II
instruments    Australia-ASSA, SSRT
Name: 4179, dtype: object

In [12]:
burst_list.loc[4179+1]

date                                                    20230124
time                                                 04:01-04:08
type                                                         III
instruments    Australia-ASSA, INDIA-OOTY, INDIA-UDAIPUR, IND...
Name: 4180, dtype: object

In [13]:
burst_list = fix_typos_in_time(burst_list)
burst_list = extract_time(burst_list)
burst_list = fix_24_hour_time(burst_list)
burst_list = create_datetime(burst_list)

In [14]:
burst_list[impossible_times_bool]

,date,time,type,instruments,time_start,time_end,date_start,date_end,datetime_start,datetime_end
1436,20220210,06:06-06:08,V,"ALMATY, Australia-ASSA, INDIA-OOTY, INDIA-UDAI...",06:06,06:08,20220210,20220210,2022-02-10 06:06:00,2022-02-10 06:08:00
1981,20220421,14:32-14:33,III,"AUSTRIA-MICHELBACH, Arecibo-Observatory, GLASG...",14:32,14:33,20220421,20220421,2022-04-21 14:32:00,2022-04-21 14:33:00
3619,20220926,23:59-00:00,III,"ALASKA-COHOE, ALASKA-HAARP, Australia-ASSA",23:59,00:00,20220926,20220927,2022-09-26 23:59:00,2022-09-27 00:00:00
3633,20220929,00:00-00:00,NaN,CTM,00:00,00:00,20220929,20220930,2022-09-29 00:00:00,2022-09-30 00:00:00
3663,20221001,03:50-00:00,CTM,*,03:50,00:00,20221001,20221002,2022-10-01 03:50:00,2022-10-02 00:00:00
5394,20230510,21:18-21:19,III,"(ALASKA-ANCHORAGE), ALASKA-COHOE, ALASKA-HAARP...",21:18,21:19,20230510,20230510,2023-05-10 21:18:00,2023-05-10 21:19:00


In [15]:
burst_list = explode_instruments_long_clean_instruments(burst_list)
burst_list = keep_only_type_I_to_VI(burst_list)

## Fix wrong names

In [16]:
MIN_BURST_PER_INSTRUMENT = 5

In [17]:
low_appearance_instruments = burst_list.groupby('instruments').filter(lambda x: len(x) <= MIN_BURST_PER_INSTRUMENT).instruments.unique().copy()
low_appearance_instruments

array(['INDIA-UAIPUR', 'ROSWELL-NW', 'DENMARK. GLASGOW', 'INPE?', 'MRT',
       'INDIA-GAURI?', 'MRO?', 'AUSTRTIA-MICHELBACH', '/INDIA-UDAIPUR',
       'l MONGOLIA-UB', 'SWISS-Landschlach', 'HUMAIN. SWISS-Landschlacht',
       'GASGOW', 'GLSAGOW', 'INDOENSIA', 'DENMAARK', 'Humain',
       'SWISS-LandschlachtEGYPT-Alexandria', 'INDIA-UDAIPUR MRT1',
       'NORWAY-NY-AALESUND', 'SP', 'IAIN-PERALEJOS', 'HUAMAIN',
       'NDIA-GAURI', 'HUMAIn', 'MRT1?', 'HUMAI', 'NDIA-UDAIPUR',
       'LASKA-COHOE', 'NDIA-OOTY', 'MEXARFT', 'POLAND', 'USTRIA-UNIGRAZ',
       'INDOALASKA-COHOE', 'ROSWELL-NMNESIA',
       'Australia-ASSA {followed by blackout}', 'SSRTMalaysia-Banting',
       'INDONESIAINDIA-OOTY', 'ALMYTY', 'INDIA-OOTY?', 'SSRTFIN',
       'LAND-Siuntio', 'USA-ARIZONA-ERA', 'FINLAND-Siunti', 'SWISS-SCAN',
       '*', 'WISS-HEITERSWIL', 'RWANDA', 'MEXICO-LANCE?', 'SSR',
       'MONGOLIA-UB MRO', 'AUSTRIA-KRUMBACH', 'Australia-ASSAG',
       'MEXICO-FCFM-UANL'], dtype=object)

In [18]:
burst_list.instruments.unique()

array(['Australia-ASSA', 'AUSTRIA-UNIGRAZ', 'HUMAIN', 'MRT1',
       'SOUTHAFRICA-SANSA', 'SWISS-Landschlacht', 'TRIEST', 'INDIA-GAURI',
       'INDIA-OOTY', 'GREENLAND', 'MEXART', 'ROSWELL-NM', 'KRIM',
       'GLASGOW', 'ALASKA-HAARP', 'ALMATY', 'INDIA-UDAIPUR', 'MRO',
       'INDIA-UAIPUR', 'INDONESIA', 'AUSTRIA-OE3FLB', 'HURBANOVO',
       'MONGOLIA-UB', 'MRT2', 'SWISS-HB9SCT', 'SWISS-IRSOL',
       'SWISS-MUHEN', 'SPAIN-PERALEJOS', 'BIR', 'DENMARK',
       'SWISS-HEITERSWIL', 'KASI', 'ALGERIA-CRAAG', 'SPAIN-ALCALA',
       'ROSWELL-NW', 'ALASKA-COHOE', 'AUSTRIA-MICHELBACH',
       'Australia-LMRO', '', 'INDIA-NASHIK', 'MRT3', 'DENMARK. GLASGOW',
       'SWISS-BLEN5M', 'SWISS-BLEN7M', 'URUGUAY', 'INPE', 'INPE?',
       'SPAIN-SIGUENZA', 'POLAND-Grotniki', 'MRT', 'GERMANY-DLR',
       'SWISS-BLEN7M-E', 'EGYPT-Alexandria', 'NORWAY-RANDABERG',
       'INDIA-GAURI?', 'SRI-Lanka', 'MRO?', 'AUSTRTIA-MICHELBACH',
       '/INDIA-UDAIPUR', 'l MONGOLIA-UB', 'SWISS-Landschlach',
       'Malays

In [19]:
def find_closest_instrument(instrument):
    """
    This function finds the closest instrument name match from a list,
    avoiding instruments in low_appearance_instruments.

    Parameters:
    - instrument (str): The name of the instrument you want to find a close match for.

    Returns:
    - str or pd.NA: Returns the closest matching instrument name or NaN if no match is found.
    """
    
    try:
        # Retrieve the unique list of instruments from the 'burst_list' dataframe.
        instruments = burst_list.instruments.unique().tolist()
        
        # Convert the list of instruments to a numpy array, excluding those 
        # present in the 'low_appearance_instruments' list.
        instruments = np.array([i for i in instruments if i not in low_appearance_instruments])
        
        # Use the 'get_close_matches' function from 'difflib' to find the closest match 
        # to the provided 'instrument' from the filtered list of 'instruments'. 
        # The function is set to return only one match with a similarity cutoff of 0.8.
        close_instrument = difflib.get_close_matches(instrument, instruments, n=1, cutoff=0.8)[0]
        
        # Print the found match for the user.
        print(f"Looking for a close match for {instrument}. Found {close_instrument}")
        return close_instrument
    
    except IndexError:
        # This block is reached if 'get_close_matches' does not find a suitable match 
        # (i.e., no matches above the cutoff of 0.8 similarity).
        print(f"Could not find a close match for {instrument}. Returning NaN.")
        return pd.NA


In [20]:
burst_list['instruments'] = burst_list['instruments'].apply(lambda x: find_closest_instrument(x) if x in low_appearance_instruments else x)

Looking for a close match for INDIA-UAIPUR. Found INDIA-UDAIPUR
Looking for a close match for INDIA-UAIPUR. Found INDIA-UDAIPUR
Looking for a close match for ROSWELL-NW. Found ROSWELL-NM
Looking for a close match for ROSWELL-NW. Found ROSWELL-NM
Could not find a close match for DENMARK. GLASGOW. Returning NaN.
Looking for a close match for INPE?. Found INPE
Looking for a close match for MRT. Found MRT3
Looking for a close match for INDIA-GAURI?. Found INDIA-GAURI
Looking for a close match for MRO?. Found MRO
Looking for a close match for MRO?. Found MRO
Looking for a close match for MRO?. Found MRO
Looking for a close match for AUSTRTIA-MICHELBACH. Found AUSTRIA-MICHELBACH
Looking for a close match for AUSTRTIA-MICHELBACH. Found AUSTRIA-MICHELBACH
Looking for a close match for /INDIA-UDAIPUR. Found INDIA-UDAIPUR
Looking for a close match for l MONGOLIA-UB. Found MONGOLIA-UB
Looking for a close match for l MONGOLIA-UB. Found MONGOLIA-UB
Looking for a close match for SWISS-Landschlach. F

In [21]:
burst_list[burst_list['instruments'].isna()]

,date,time,type,instruments,time_start,time_end,date_start,date_end,datetime_start,datetime_end
982,20210522,10:22-10:22,III,<NA>,10:22,10:22,20210522,20210522,2021-05-22 10:22:00,2021-05-22 10:22:00
6471,20220306,15:46-15:56,VI,<NA>,15:46,15:56,20220306,20220306,2022-03-06 15:46:00,2022-03-06 15:56:00
6492,20220307,13:36-13:36,III,<NA>,13:36,13:36,20220307,20220307,2022-03-07 13:36:00,2022-03-07 13:36:00
6497,20220307,14:50-14:50,III,<NA>,14:50,14:50,20220307,20220307,2022-03-07 14:50:00,2022-03-07 14:50:00
6605,20220309,13:19-13:19,III,<NA>,13:19,13:19,20220309,20220309,2022-03-09 13:19:00,2022-03-09 13:19:00
7076,20220328,11:23-11:38,II,<NA>,11:23,11:38,20220328,20220328,2022-03-28 11:23:00,2022-03-28 11:38:00
7113,20220328,12:49-12:50,III,<NA>,12:49,12:50,20220328,20220328,2022-03-28 12:49:00,2022-03-28 12:50:00
15104,20220723,20:33-20:35,III,NaN,20:33,20:35,20220723,20220723,2022-07-23 20:33:00,2022-07-23 20:35:00
17273,20220920,11:20-11:24,III,<NA>,11:20,11:24,20220920,20220920,2022-09-20 11:20:00,2022-09-20 11:24:00
17413,20220921,10:06-10:08,III,<NA>,10:06,10:08,20220921,20220921,2022-09-21 10:06:00,2022-09-21 10:08:00


In [22]:
burst_list = burst_list.dropna(subset=['instruments'])

In [23]:
burst_list['type'] = burst_list['type'].replace({'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5, 'VI': 6}).astype(int)

In [24]:
burst_list[burst_list.instruments.str.contains('Australia-ASSA') & burst_list.type.isin([5, 6])]

,date,time,type,instruments,time_start,time_end,date_start,date_end,datetime_start,datetime_end
113,20210420,03:19-03:21,6,Australia-ASSA,03:19,03:21,20210420,20210420,2021-04-20 03:19:00,2021-04-20 03:21:00
168,20210423,07:16-07:16,6,Australia-ASSA,07:16,07:16,20210423,20210423,2021-04-23 07:16:00,2021-04-23 07:16:00
332,20210506,03:00-03:04,6,Australia-ASSA,03:00,03:04,20210506,20210506,2021-05-06 03:00:00,2021-05-06 03:04:00
492,20210512,05:43-05:46,6,Australia-ASSA,05:43,05:46,20210512,20210512,2021-05-12 05:43:00,2021-05-12 05:46:00
534,20210513,06:28-06:30,6,Australia-ASSA,06:28,06:30,20210513,20210513,2021-05-13 06:28:00,2021-05-13 06:30:00
...,...,...,...,...,...,...,...,...,...,...
36629,20230908,21:21-21:36,6,Australia-ASSA,21:21,21:36,20230908,20230908,2023-09-08 21:21:00,2023-09-08 21:36:00
36636,20230908,21:49-22:12,6,Australia-ASSA,21:49,22:12,20230908,20230908,2023-09-08 21:49:00,2023-09-08 22:12:00
36676,20230909,07:16-07:26,6,Australia-ASSA,07:16,07:26,20230909,20230909,2023-09-09 07:16:00,2023-09-09 07:26:00
36759,20230911,01:35-02:07,6,Australia-ASSA,01:35,02:07,20230911,20230911,2023-09-11 01:35:00,2023-09-11 02:07:00


In [28]:
burst_list.to_excel('burst_list.xlsx', index=False)

# Manually fix the rest. 